# Deep Explainer

这一章节简单记录下常用的模型解释工具 SHAP 的用法。

主要参考了如下资料：

- [Push the limits of explainability — an ultimate guide to SHAP library](https://t.co/wf4Z7B1MYd?amp=1)
- [Interpreting your deep learning model by SHAP](https://towardsdatascience.com/interpreting-your-deep-learning-model-by-shap-e69be2b47893)
- [Case study: explaining credit modeling predictions with SHAP](https://t.co/uTHcXwaddU?amp=1)
- [Interpreting complex models with SHAP values](https://medium.com/@gabrieltseng/interpreting-complex-models-with-shap-values-1c187db6ec83)

## 一点常识

首先补充一些常识性知识。

在水文水资源等领域，我们不仅只关注模型的结果，对于模型决策的跟踪和解释也很重要。

首先，区分几个概念：

- Explainability: 从技术角度解释算法预测的可能性
- Interpretability: 用人类可以理解的术语解释或提供意义的能力
- Transparency: 如果模型本身是可以理解的，那么它就是透明的。

为什么关注 Interpretability？

可解释性有助于确保决策过程的公正性，即检测并纠正训练数据集中的偏差。此外，它通过突出可能改变预测的潜在不利干扰来增强鲁棒性。它还可以确保只有重要特征才能推断出结果。

有时候，最好放弃机器学习方法，而使用基于行业知识或法规证明的规则的确定性算法。

但是，访问具有高准确性的机器学习算法的功能太诱人了。可以考虑准确性和可解释性之间的权衡。"权衡"的意思如下。

考虑一个线性回归模型：

$$f(x_1,x_2,\cdots,x_n)=\phi _1x_1+\phi _2x_2+\cdots+\phi _nx_n$$

这时候要知道某一个特征的重要性是很容易的，即有最大绝对值的系数$\phi _i$的那个$x_i$。但是模型太简单，功能不够强,所以需要一个能表达非线性的复杂模型.

但是当使用了复杂模型时,就很难像线性模型那样很明确的知道重要的影响因素了.

为了实现这些目标，出现了一个新领域：XAI（可解释人工智能），旨在产生既强大又可解释的算法。

已经提出了许多框架来帮助解释非透明算法。在[Cloudera白皮书](https://ff06-2020.fastforwardlabs.com/?utm_campaign=Data_Elixir&utm_source=Data_Elixir_282)中很好地介绍了这些方法。

实践中，SHapley Additive exPlantions (SHAP)就是最常用的框架之一。并且近来，可解释性工具变得越来越容易使用。例如，机器学习平台Dataiku在2020年3月2日发布的最新版本7.0中添加了可解释性工具;Azure ML提出了自己的Shap版本和添加交互式仪表板的工具。在[这篇文章](https://t.co/rsYCcwshqO?amp=1)中也描述了这样的开源Web应用程序，它有助于探索SHAP库。这些工具非常有趣，可以快速获得解释的概述。下面就记录下SHAP的基本情况。

## 什么是SHAP

SHAP（SHapley Additive exPlantions）是一种为给定预测构建特征重要性的线性解释模型的方法(后面有一些解释)。它适用于任何模型，甚至是非线性模型，如深度学习神经网络。

某些模型类型（例如逻辑回归或树）具有快速精确的版本，而其他模型类型则在近似值，速度和采样变异性之间进行权衡。

SHAP之所以具有吸引力，是因为它基于博弈论中的Shapley值，这是满足[Lundberg等人](https://proceedings.neurips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf)，即SHAP的作者，提出的以下三个有用特性的解释：

1. 局部精度（大致上：解释模型与所解释的点处的真实模型匹配）
2. 缺失（伦德伯格：“缺失要求原始输入中缺少的要素没有影响”）
3. 一致性（Lundberg：“无论什么时候我们更改模型它都依赖于某个功能，那么该功能的重要性就不应降低。”）

Lundberg[说](https://towardsdatascience.com/interpretable-machine-learning-with-xgboost-9ec80d148d27)，一致性使我们能够有意义地比较两个模型之间的特征重要性。

本文不详细解释库背后的计算是如何完成的。网上有许多资源，例如[SHAP文档](https://shap.readthedocs.io/en/stable/index.html)，[《可解释性机器学习》](https://christophm.github.io/interpretable-ml-book/)等。这里仅仅记录一些最基本的原理。

回顾线性模型的解释可以为非线性模型的解释提供思路, 可以为每个特征$x_i$线性地分配一个$\phi _i$ ,它线性地描述了特征如何影响模型输出. 我们已经讨论了线性模型的缺点，但是：在许多数据点上，系数$\phi$将无法捕获复杂的关系。但在单个级别上，它们会做得很好，因为对于单个预测，每个变量将以恒定值影响模型的预测。

例如，分析一家人中谁喜欢计算机游戏, 影响因素有工作,年龄,性别等, 以50岁的大叔Frank为例，他是50岁的视频游戏测试员。对他来说，$\phi_{Job}$会很高，而$\phi_{Age}$会很低。但是，对于14岁的少年Bobby来说，$\phi_{Age}$会很高。

也就是说,我们的预测采用了一个复杂的模型，该模型学习了数据中的非线性模式，我们可以将其分解为许多描述单个数据点的线性模型来解释。

重要的是要注意，这些解释系数$\phi$不是模型的输出，而是我们用来解释该模型的内容。通过将所有这些简单的单个模型汇总在一起，我们可以了解该模型在所有样例中的行为方式。

因此，总结一下：

与其尝试解释整个复杂模型，不如尝试解释复杂模型在一个数据点上的表现。我将使用线性解释模型进行此操作。我们称它为g。

另外，为了进一步简化目前的简单模型来考虑问题，不将系数$\phi$与原始特征值x相乘。相反，如果存在该功能，则将其乘以1，否则将其乘以0。

那么在预测谁喜欢计算机游戏的例子中，我得到的是以下内容：

$$g_{Frank}=\phi _{FrankAge}+\phi _{FrankGender}+\phi _{FrankJob}$$

其中g_Frank = p_Frank，即模型对弗兰克的原始预测。

注意，这些系数仅适用于Frank。如果要查找Bobby模型的行为，则需要找到一组新的系数。另外，由于Bobby没有工作，将$\phi_{Bobby Job}$乘以0。因此，他的简单模型将是

$$g_{Bobby}=\phi_{BobbyAge}+\phi_{BobbyGender}$$

将对所有数据点进行此操作并将其汇总，以了解模型如何在全局范围内工作。

现在，在此框架下，就能解释复杂的模型

那么如何计算 $\phi$值. 简而言之，通过比较使用和不使用某特征的模型预测值，Shapley's value（如下公式） 计算了特征的重要性。这种方法是受博弈论启发的：

$$\phi _i (v)=\frac{1}{|N|!}\sum _R[v(P^R_i \cup {i})-v(P^R_i)]$$

其中，$\phi$是Shapely value，N是feature（player）的个数，$P^R_i$是有顺序的player集合，$v(P^R_i)$是有顺序的player集合的贡献，$v(P^R_i \cup {i})$有顺序的player集合和第i个player的贡献。

再举个例子。我们的目标是提供一个深度学习模型，该模型需要完成100行代码，我们有3位数据科学家（L，M，N）。他们3个必须一起工作才能交付项目，下图是三人贡献情况:

![](pictures/1_ArZVijQ-I_nNpn-0pukSpg.png)

那么可以计算不同顺序下边缘贡献，三个player，那么所有可能排列情况就是3！=6：

![](pictures/1_DLL5sCQKeVXboAYIvdgwUw.png)

那么每个contributor的贡献如下：

![](pictures/1_uGjQRe9U0ebC5HxYXAzg3A.png)

可以看到尽管M的能力是N的6倍（30对5），但M获得41.7％的reward，而N能获得24.17％的reward。

换到解释模型中，所有特征都是contributor，并试图预测任务，即game，reward是实际预测减去解释模型的结果。

在SHAP中，将特征重要性分配给每个特征，这等同于所提到的contribution。以汽车贷款为例。有“新司机”，“有孩子”，“四门”和“年龄”。

从理论上讲，组合数为2 ^ n，其中n为特征数。鉴于我们想知道“年龄”的Shapley值。我们将预测以下具有“年龄”功能和不具有“年龄”功能的所有组合:

![](pictures/1_fdqZ1XivRBZzuuvyv8yvhw.png)

通过使用Shapley公式，SHAP将计算以上所有情况并返回平均贡献和。注意这不是在谈论特定功能缺失时的区别。

可以想到如果遍历所有的features,很容易就计算不可行了.但是SHAP提供了一些优化方法使得能在实际中应用这类解释方法. 

## SHAP 框架

SHAP有一套非常复杂的可视化效果，即使对于数据科学家而言，也并非总是直观的。SHAP用于解释现有模型，其流程如下图所示。以使用sklearn模型构建的二进制分类案例为例。我们训练，调整和测试模型。然后，可以使用我们的数据和模型来创建一个额外的SHAP模型，以解释我们的分类模型。

![](pictures/0_jI6QkeJnMrVGrGY6.png)

下面是构成SHAP的基本模块。

- global explanations：从一般的角度解释模型的工作方式
- local explanations：样本模型（数据点）的说明
- explainer （shap.explainer_type（params））：根据所使用的模型选择的可解释性算法的类型。对于每种类型的模型，参数都不同。通常，必须至少提供模型和训练数据。
- base value（explainer.expected_value）：E（y_hat）是“如果我们不知道当前输出的任何特征将被预测的值”，是训练数据集或背景集的均值（y_hat）预测。我们可以称其为“参考值”，它是一个标量（n）。如下图所示的形式 ![](pictures/1_hIGUjhaoPX-mA9L4MO5H8g.png)
- SHAPley values（explainer.shap_values（x））：基于所有可能的特征，每个特征对每个样本的每个预测的平均贡献。它是一个（n，m），即n个样本，m个特征的矩阵，表示每个特征对每个样本的贡献。
- output value（对于样本）：由算法预测的值（模型的概率，对数或原始输出值）
- display features (n x m)：在特征的变换/编码/工程化等之前原始值的矩阵，可以将其提供给某些图形以改善解释。

其中，Shapley values然是中心要素。这是一个与我们的输入数据具有相同维度的矩阵，并且我们可以用不同的方式对其进行分析以解释该模型。我们可以减小其尺寸，可以对其进行聚类，也可以使用它来创建新功能。

### SHAP 可视化

SHAP 库通过可视化提出了丰富但并非详尽的探索。[这篇文章](https://francescopochetti.com/whitening-a-black-box-how-to-interpret-a-ml-model/)介绍了用自编码和SHAP来提高异常检测的能力。

SHAP库提供了不同的可视化效果。

- The summary plot：显示了最重要的特征及其对模型的影响程度。它可以采用几种图形形式，对于TreeExplainer解释的模型，还可以使用“紧凑点”和输入中的shap_interaction_values来观察 interaction values。
- The dependency plot：通过建议一个概率观察相互作用来对特征进行二乘二分析。散点图表示特征（x）和由第二个特征（hue）着色的shapley值（y）之间的依赖性。

![](pictures/1_mP_7ZW9j_dDWaEGVKdMB8Q.png)

同时观察三因素关系可能并不直观。通过观察颜色观察依存关系是否科学上是准确的也存疑。Shap可以为我们提供一个交互关系，该关系被计算为第一要素的shapley值和第二要素的值之间的相关性。如果可能的话（对于TreeExplainer），使用shapley交互值观察交互更有意义。

除了global explanation，在local explanation方面，可视化的样本解释有三种选择：force plot, decision plot 和 waterfall plot。

![](pictures/1_LKpKHFP3N7Toj_lt_uY12g.png)

三个表示以非常相似的方式表示信息。同时，这些图的某些元素是互补的。通过将三者并排放置，有一种更直观的方式来理解结果。force plot 可以很好地看出“output value”相对于“base value”的位置。我们还可以看到哪些特征对预测有正面（红色）或负面（蓝色）影响，以及这种影响的程度。waterfall plot 使我们能够量化查看特征影响的幅度和性质。它还可以查看特征重要性的顺序以及每个特征对研究样本的取值。 decision plot使观察每个变化的幅度成为可能

通过使用force图和decision图，我们可以同时表示多个样本。可以将一组样本的force图与树状图的最后一级进行比较。样本按相似性或所选特征分组。我认为，很难从随机样本中读取该图。如果我们代表不同的案例或背后有一个假设，那将更有意义。

如果我们选择太多的样本，那么对于一组样本的决策图很快就会变得很笨重。观察一组有限样本的“轨迹偏差”或“发散/收敛轨迹”非常有用。

### SHAP Explainers 

Explainers 解释器 是用于计算Shapley值的模型。下图显示了不同类型的Explainers。

![](pictures/1_FJ2Ay_MmhzYKamAzCdvPAg.png)

Explainers 的选择主要取决于所选的学习模型。对于线性模型，使用“线性解释器”，对于决策树和“set”类型模型使用“TreeExplainer”。“Kernel Explainer”比上述解释器慢。

另外，“TreeExplainer”允许显示交互值。它还允许将模型输出转换为概率或对数损失，这对于更好地理解模型或比较多个模型很有用。

Kernel Explainer 会创建一个模型，该模型替代最接近我们的模型。Kernel Explainer可用于解释神经网络。

对于深度学习模型，有Deep Explainer和Grandient Explainer。